# Regression

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import sklearn

# To create test/train splits
from sklearn.model_selection import train_test_split
# To help us go through different parameter configurations for
# each type of model.
from sklearn.model_selection import GridSearchCV
# To help us evaluate the model on each trial or "split"
from sklearn.model_selection import cross_val_score

# Naive Bayes
from sklearn.neighbors import KNeighborsRegressor
# Neural Networks
from sklearn.neural_network import MLPRegressor

# Confusion matrix
from sklearn.metrics import confusion_matrix

In [13]:
# Import our regression evaluation metrics
from sklearn.metrics import explained_variance_score, r2_score

In [14]:
# This is to stop the barrage of warning messages we'll get later
import warnings; warnings.simplefilter('ignore')

In [15]:
# Create a seed to use later.
seed = 42

In [16]:
# First import your dataset as usual
dataset = "datasets/winequality_red.csv"

df = pd.read_csv(dataset, delimiter=";")

df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [17]:
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [18]:
# Let's see how big our dataset is
n_samples, n_columns = df.shape
print("Number of samples: {}".format(n_samples))

Number of samples: 1599


In [19]:
# Step 1: Create visualization

# For this dataset, we can't get it to plot easily, we'll try this later

In [20]:
# Step 2: Separate features and labels
df_X = df.drop("quality", axis=1)
df_X.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


In [21]:
df_y = df["quality"]
df_y.head()

0    5
1    5
2    5
3    6
4    5
Name: quality, dtype: int64

In [22]:
# Step 3: Split into test, train
# We want 70% train, 30% test
train_X, test_X, train_y, test_y = train_test_split(df_X, df_y,
                                                    test_size=0.3,
                                                    random_state=seed)

## Step 4: Train Estimators

To demonstrate how to do crossfold-validation (CV), we'll use two algorithms:
- [Neural Network](http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html)
- [KNN](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html)


Here's what we'll be doing:
- Defining a "grid" of possibilities for the models' parameters
- Training on "splits" of our data
- Keeping the best model of each type (KNN, NN) that performed best on our dataset
- Showing the accuracies and confusion matrices of those best-performing models

__JUPYTER PRO-TIP__: You can time the run-time of individual cells in Jupyter by putting `%%time` at the start of any cell.

In [23]:
%%time
# We're going to build nn_params, a list of dictionaries
# Each dictionary has the variable name to try modifying, and
# which values to try for it.
# All combinations of variables in the dictionary below will be tried

nn_params = [
    {
        "hidden_layer_sizes": [
            (20,), (30,), # models with 1 hidden layer
        ],
        "max_iter": [40, 100],
        "activation": ["logistic", "tanh", "relu"],
        "learning_rate_init": [1e-4, 1e-3],
    },
]

# nn_tester will test the permutations of parameters in nn_params
# using 5 trials (splitting train set into 5, training on 4 and testing
# on the 5th).
nn_model = MLPRegressor(random_state=seed)
nn_experimenter = GridSearchCV(nn_model, nn_params, cv=5)
nn_experimenter.fit(train_X, train_y)

print("Best parameter set found: ")
print(nn_experimenter.best_params_)

Best parameter set found: 
{'activation': 'tanh', 'hidden_layer_sizes': (20,), 'learning_rate_init': 0.001, 'max_iter': 100}
CPU times: user 16.8 s, sys: 2.61 s, total: 19.4 s
Wall time: 18.6 s


## Step 5: Evaluation for Regression

Evaluating regression problems are slightly different, we use two different metrics for this:
- [explained_variance_score]() -- best possible score is `1.0`, lower values are worse
- [r2_score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.explained_variance_score.html#sklearn.metrics.explained_variance_score) -- best possible score is `1.0`, lower values are worse

We will use just `r2_score()` below for now but `explained_variance_score()` works similarly.

In [27]:
train_preds = nn_experimenter.predict(train_X)
test_preds = nn_experimenter.predict(test_X)

train_r2 = r2_score(train_preds, train_y)
test_r2 = r2_score(test_preds, test_y)

print("Train R2 score: {:.1f}%".format(train_r2))
print("Test R2 score {:.1f}%".format(test_r2))

Train R2 score: -4.3%
Test R2 score -4.3%


In [28]:
%%time

# Now we can do the same thing with KNN

# Notice here how we decide we only want one dictionary of params
# to try all permutations of.
knn_params = [
    {
        "n_neighbors": [2, 3, 4, 5, 8],
        "p": [1, 2],
        "algorithm": ["ball_tree", "kd_tree"]
    },
]

knn_model = KNeighborsRegressor()
knn_exp = GridSearchCV(knn_model, knn_params, cv=5)

knn_exp.fit(train_X, train_y)

print("Best parameter set found: ")
print(knn_exp.best_params_)

Best parameter set found: 
{'algorithm': 'ball_tree', 'n_neighbors': 8, 'p': 1}
CPU times: user 790 ms, sys: 7.25 ms, total: 797 ms
Wall time: 803 ms


In [29]:
train_preds = knn_exp.predict(train_X)
test_preds = knn_exp.predict(test_X)

train_r2 = r2_score(train_preds, train_y)
test_r2 = r2_score(test_preds, test_y)

print("Train R2 score: {:.1f}%".format(train_r2))
print("Test R2 score {:.1f}%".format(test_r2))

Train R2 score: -1.2%
Test R2 score -2.1%


In [40]:
best_knn = KNeighborsRegressor()
best_knn.set_params(**knn_exp.best_params_)

KNeighborsRegressor(algorithm='ball_tree', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=8, p=1,
          weights='uniform')

In [43]:
knn_exp.best_estimator_

KNeighborsRegressor(algorithm='ball_tree', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=8, p=1,
          weights='uniform')

In [46]:
best_knn.fit(train_X, train_y)

KNeighborsRegressor(algorithm='ball_tree', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=8, p=1,
          weights='uniform')

In [47]:
trn_knn = best_knn.predict(train_X)
test_knn =  best_knn.predict(test_X)

In [ ]:
assert trn_knn == train_preds
assert tst_knn == 